**Dealing with duplicated values**

In [90]:
import pandas as pd

df = pd.read_csv('/content/sample_data/part1.csv', index_col='index')
df_copy = df.copy()

In [91]:
df_copy.head()

,order id,date,status,fulfilment,sales channel,ship-service-level,style,sku,category,size,asin,courier status,qty,currency,amount,ship-city,ship-state,ship-postal-code,ship-country,b2b
index,,,,,,,,,,,,,,,,,,,,
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,Unknown,0,INR,647.62,MUMBAI,MAHARASHTRA,400081,IN,False
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085,IN,False
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210,IN,True
3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,Unknown,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008,IN,False
4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073,IN,False


Let's check how many unique values are in DataFrame

In [92]:
df_copy.nunique()

,0
order id,120378
date,91
status,13
fulfilment,2
sales channel,2
ship-service-level,2
style,1377
sku,7195
category,9
size,11


The first step was to determine the **grain** of the dataset — that is, what a single row represents.

In [93]:
is_order_grain = df_copy['order id'].nunique() / len(df_copy) * 100
is_order_grain

93.3343671254119

The result was 93.33%, meaning that approximately 6.67% of rows share the same order id.

In [94]:
df_copy['order id'].value_counts().sort_values(ascending=False)

,count
order id,
403-4984515-8861958,12
171-5057375-2831560,12
403-0173977-3041148,11
404-9932919-6662730,11
408-3317403-1729937,10
...,...
406-1589049-9362704,1
403-0275824-1655546,1
404-4885091-8361169,1


As we can see, we have duplicated order_id's - one order may have multiple rows. <br>Grain != Order.<br> Let's analyze what happens during filtering by one order_id.

In [102]:
df_copy[df_copy['order id'] == '403-4984515-8861958']

,order id,date,status,fulfilment,sales channel,ship-service-level,style,sku,category,size,asin,courier status,qty,currency,amount,ship-city,ship-state,ship-postal-code,ship-country,b2b
31736,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,JNE3792,JNE3792-KR-XXXL,kurta,3XL,B09KXNWRSB,Shipped,1,INR,432.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31737,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,JNE3764,JNE3764-KR-XXXL,kurta,3XL,B09K3SSDSZ,Shipped,1,INR,487.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31738,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,JNE3503,JNE3503-KR-XXXL,kurta,3XL,B08RP2BV5Q,Shipped,1,INR,318.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31739,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,J0308,J0308-DR-XXXL,Western Dress,3XL,B099NTV5ZV,Shipped,1,INR,665.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31740,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,JNE3702,JNE3702-KR-XXXL,kurta,3XL,B093ZRBGR9,Shipped,1,INR,342.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31741,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,JNE3741,JNE3741-KR-XXXL,kurta,3XL,B099NL77RF,Shipped,1,INR,432.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31742,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,JNE3634,JNE3634-KR-XXXL,kurta,3XL,B097ZZTMZD,Shipped,1,INR,511.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31743,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,SET433,SET433-KR-NP-XXXL,Set,3XL,B09TH8T4GM,Shipped,1,INR,666.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31744,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,JNE3636,JNE3636-KR-XXXL,kurta,3XL,B09812FYH9,Shipped,1,INR,459.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31745,403-4984515-8861958,04-11-22,Shipped,Amazon,Amazon.in,Expedited,JNE3633,JNE3633-KR-XXXL,kurta,3XL,B094FQ32VX,Shipped,1,INR,459.0,SOLAPUR,MAHARASHTRA,413002,IN,False


Order id may be the same but we have other different values. Ex: sku. One order - many different products.

In [96]:
df_copy.duplicated().sum() # 6 rows are full duplicates of other rows

np.int64(6)

In [97]:
df_copy[df_copy.duplicated(subset=['order id'], keep=False)].sort_values(by='order id')

,order id,date,status,fulfilment,sales channel,ship-service-level,style,sku,category,size,asin,courier status,qty,currency,amount,ship-city,ship-state,ship-postal-code,ship-country,b2b
index,,,,,,,,,,,,,,,,,,,,
78939,171-0012693-2893106,05-07-22,Cancelled,Merchant,Amazon.in,Standard,NW015,NW015-TP-PJ-S,Set,S,B0922SCDFN,Unknown,0,INR,477.14,SRI GANGANAGAR,RAJASTHAN,335001,IN,False
78938,171-0012693-2893106,05-07-22,Cancelled,Merchant,Amazon.in,Standard,NW030,NW030-TP-PJ-S,Set,S,B09G2RFP14,Unknown,0,INR,538.10,SRI GANGANAGAR,RAJASTHAN,335001,IN,False
91476,171-0014231-7760344,06-29-22,Pending,Amazon,Amazon.in,Expedited,J0224,J0224-SKD-XS,Set,XS,B09RKFBZZD,Unshipped,1,INR,1364.00,AHMEDABAD,Gujarat,380015,IN,False
91475,171-0014231-7760344,06-29-22,Pending,Amazon,Amazon.in,Expedited,SET342,SET342-KR-NP-N-XS,Set,XS,B09Q1943PQ,Shipped,1,INR,850.00,AHMEDABAD,Gujarat,380015,IN,False
24515,171-0015435-1981904,04-16-22,Shipped,Amazon,Amazon.in,Expedited,JNE3611,JNE3611-KR-L,kurta,L,B08XVQMSJV,Shipped,1,INR,459.00,Pimpri Chinchwad,MAHARASHTRA,412101,IN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74748,408-9929975-2118741,05-11-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,SET408,SET408-KR-NP-XXXL,Set,3XL,B09RKF7PZB,Shipped,1,INR,499.00,port blair,ANDAMAN & NICOBAR,744101,IN,False
36052,408-9951521-7399501,04-09-22,Shipped,Amazon,Amazon.in,Expedited,J0338,J0338-DR-M,Western Dress,M,B0982ZPB81,Shipped,1,INR,744.00,SECUNDERABAD,TELANGANA,500017,IN,False
36053,408-9951521-7399501,04-09-22,Shipped,Amazon,Amazon.in,Expedited,J0335,J0335-DR-M,Western Dress,M,B09831XC46,Shipped,1,INR,807.00,SECUNDERABAD,TELANGANA,500017,IN,False


Filtering duplicates based on order_id and sku.

In [98]:
df_copy[df_copy.duplicated(subset=['order id', 'sku'], keep=False)]

,order id,date,status,fulfilment,sales channel,ship-service-level,style,sku,category,size,asin,courier status,qty,currency,amount,ship-city,ship-state,ship-postal-code,ship-country,b2b
index,,,,,,,,,,,,,,,,,,,,
30660,406-0372545-6086735,04-12-22,Cancelled,Amazon,Amazon.in,Expedited,SET197,SET197-KR-NP-L,Set,L,B08B3YPD63,Cancelled,0,INR,648.56,Siliguri,WEST BENGAL,734008,IN,False
30661,406-0372545-6086735,04-12-22,Cancelled,Amazon,Amazon.in,Expedited,SET197,SET197-KR-NP-L,Set,L,B08B3YPD63,Cancelled,0,INR,648.56,Siliguri,WEST BENGAL,734008,IN,False
41291,408-0373839-4433120,04-05-22,Cancelled,Amazon,Amazon.in,Expedited,JNE3501,JNE3501-KR-M,kurta,M,B08MYVCDB4,Cancelled,0,INR,648.56,BENGALURU,KARNATAKA,560072,IN,False
41292,408-0373839-4433120,04-05-22,Cancelled,Amazon,Amazon.in,Expedited,JNE3501,JNE3501-KR-M,kurta,M,B08MYVCDB4,Cancelled,0,INR,648.56,BENGALURU,KARNATAKA,560072,IN,False
52330,407-8364731-6449117,05-29-22,Shipped,Amazon,Amazon.in,Expedited,JNE3769,JNE3769-KR-L,kurta,L,B09K3T675B,Cancelled,0,INR,648.56,Theni,TAMIL NADU,625579,IN,False
52333,407-8364731-6449117,05-29-22,Shipped,Amazon,Amazon.in,Expedited,JNE3769,JNE3769-KR-L,kurta,L,B09K3T675B,Shipped,1,INR,487.00,Theni,TAMIL NADU,625579,IN,False
79844,171-9628368-5329958,05-07-22,Cancelled,Amazon,Amazon.in,Expedited,J0329,J0329-KR-L,kurta,L,B09KXRB7JV,Cancelled,0,INR,648.56,ERNAKULAM,KERALA,682017,IN,False
79845,171-9628368-5329958,05-07-22,Cancelled,Amazon,Amazon.in,Expedited,J0329,J0329-KR-L,kurta,L,B09KXRB7JV,Cancelled,0,INR,648.56,ERNAKULAM,KERALA,682017,IN,False
85790,171-3249942-2207542,05-03-22,Shipped,Amazon,Amazon.in,Expedited,SET323,SET323-KR-NP-XL,Set,XL,B09NDKKM7P,Shipped,1,INR,939.00,PUNE,MAHARASHTRA,411057,IN,False


There we can see that we have 6 exact duplicates that we can get removed but also a business case: <br>
52330 / 52333 <br>
order id = 407-8364731-6449117 <br>
sku = JNE3769-KR-L <br>
that may be a refund or shipping problem case. This row will be further analyzed.

In [99]:
df_copy = df_copy.drop_duplicates(keep='first').reset_index(drop=True)

In [100]:
#check
df_copy.duplicated(subset=['order id','sku']).sum()

np.int64(1)

In [101]:
df_copy.to_csv('sample_data/part2.csv')

Summary:
**grain**: One row represents one product (SKU) within one order.
<br>
Logical key = (order id, sku) <br>
So, grain is reprezented by one product within order with it's status. Repeated order id values are expected due to multi-product orders.
Duplicates are dropped.